# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [8]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [9]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [14]:
total_Q = data.groupby(['CustomerID','ProductName'])['Quantity'].sum()
total_Q

CustomerID  ProductName                  
33          Apricots - Dried                  1
            Assorted Desserts                 1
            Bandage - Flexible Neon           1
            Bar Mix - Pina Colada, 355 Ml     1
            Beans - Kidney, Canned            1
                                             ..
98200       Vol Au Vents                     50
            Wasabi Powder                    25
            Wine - Fume Blanc Fetzer         25
            Wine - Hardys Bankside Shiraz    25
            Yogurt - French Vanilla          25
Name: Quantity, Length: 63628, dtype: int64

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [17]:
data_pivot = pd.pivot_table(data, values='Quantity', index='ProductName', columns='CustomerID').fillna(0)
data_pivot.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0
Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [24]:
distances = pdist(data_pivot.T, 'euclidean')
print(distances)
squar_dist = squareform(distances)
print('----Squareform----')
print(squar_dist)



[ 10.72380529   9.64365076   9.89949494 ... 266.9269563  266.9269563
 269.25824036]
----Squareform----
[[  0.          10.72380529   9.64365076 ... 197.94948851 218.8241303
  203.79892051]
 [ 10.72380529   0.          10.67707825 ... 197.34994299 218.62525014
  203.95342606]
 [  9.64365076  10.67707825   0.         ... 197.32460566 218.02981447
  203.80628057]
 ...
 [197.94948851 197.34994299 197.32460566 ...   0.         266.9269563
  266.9269563 ]
 [218.8241303  218.62525014 218.02981447 ... 266.9269563    0.
  269.25824036]
 [203.79892051 203.95342606 203.80628057 ... 266.9269563  269.25824036
    0.        ]]


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [73]:
distances_df = pd.DataFrame(1/(1 + squareform(pdist(data_pivot.T, 'euclidean'))), 
                         index=data_pivot.columns, columns=data_pivot.columns)

distances_df.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.085297,0.093953,0.091747,0.087410,0.089695,0.085297,0.088913,0.088152,0.089695,...,0.004809,0.005108,0.004996,0.005421,0.004920,0.005023,0.004880,0.005026,0.004549,0.004883
200,0.085297,1.000000,0.085638,0.085297,0.080070,0.083020,0.084959,0.083651,0.085638,0.087047,...,0.004825,0.005121,0.005014,0.005448,0.004925,0.005032,0.004909,0.005042,0.004553,0.004879
264,0.093953,0.085638,1.000000,0.088152,0.089301,0.087047,0.085638,0.086333,0.087047,0.087047,...,0.004822,0.005115,0.004996,0.005441,0.004932,0.005055,0.004894,0.005042,0.004566,0.004883
356,0.091747,0.085297,0.088152,1.000000,0.085983,0.086688,0.085983,0.091325,0.085983,0.087410,...,0.004814,0.005111,0.004999,0.005437,0.004920,0.005036,0.004871,0.005042,0.004563,0.004886
412,0.087410,0.080070,0.089301,0.085983,1.000000,0.085638,0.085638,0.089301,0.084959,0.087779,...,0.004808,0.005131,0.004996,0.005441,0.004925,0.005042,0.004876,0.005039,0.004568,0.004903


In [75]:
#customers similiar to customerID 33
similarities = distances_df[33].sort_values(ascending=False)[1:6]
similarities

CustomerID
3909    0.095358
264     0.093953
3531    0.093953
2503    0.093498
3305    0.093051
Name: 33, dtype: float64

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [79]:
#total_similar = total_Q.loc[similarities.keys()]

total_similar = pd.DataFrame(total_Q.loc[similarities.keys()])
total_similar

Quantity
CustomerID ProductName                             
264        Apricots - Halves                      1
           Apricots Fresh                         1
           Bacardi Breezer - Tropical             1
           Bagel - Plain                          1
           Banana - Leaves                        1
...                                             ...
3909       Wine - Charddonnay Errazuriz           2
           Wine - Fume Blanc Fetzer               1
           Wine - Magnotta, Merlot Sr Vqa         1
           Wine - Toasted Head                    1
           Wine - Valpolicella Masi               1

[282 rows x 1 columns]

## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [81]:
buy_products = total_similar.groupby('ProductName').sum().sort_values(by='Quantity',ascending=False)
buy_products

,Quantity
ProductName,
"Salsify, Organic",4
Quiche Assorted,3
Chocolate - Dark,3
Wine - Charddonnay Errazuriz,3
Bay Leaf,3
...,...
Gatorade - Xfactor Berry,1
Gloves - Goldtouch Disposable,1
Grouper - Fresh,1


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [110]:
total = pd.concat([buy_products, total_Q.loc[33]], axis=1).fillna(0)
total.columns = ['Quantity Colegas', 'Quantity 33']
total.sort_values(by= 'Quantity Colegas', ascending=False)[total['Quantity 33']==0].head()


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until


['Chocolate - Dark',
 'Wine - Ej Gallo Sierra Valley',
 'Bay Leaf',
 'Wine - Charddonnay Errazuriz',
 'Pecan Raisin - Tarts']

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [117]:

customersID = list(data_pivot.columns)
def prod_recommend(customers):
    recommendations = dict()
    for cust in customers:
        similarities = distances_df[cust].sort_values(ascending=False)[1:6]
        total_similar = pd.DataFrame(total_Q.loc[similarities.keys()])
        buy_products = total_similar.groupby('ProductName').sum().sort_values(by='Quantity',ascending=False)
        total = pd.concat([buy_products, total_Q.loc[cust]], axis=1).fillna(0)
        total.columns = ['Quantity Colegas', 'Quantity 33']
        products = list(total.sort_values(by= 'Quantity Colegas', ascending=False)[total['Quantity 33']==0].head().T.columns)
        recommendations[cust]=products
    return recommendations
step8 = prod_recommend(customersID)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  # Remove the CWD from sys.path while we load stuff.


##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [130]:
step9 = pd.DataFrame(step8).T.reset_index().head()
step9

,index,0,1,2,3,4
0,33,Chocolate - Dark,Wine - Ej Gallo Sierra Valley,Bay Leaf,Wine - Charddonnay Errazuriz,Pecan Raisin - Tarts
1,200,Scampi Tail,Muffin - Carrot Individual Wrap,Bay Leaf,Pork - Kidney,Soup - Campbells Bean Medley
2,264,Cheese - Parmesan Grated,Soup - Campbells Tomato Ravioli,Pork - Kidney,Beer - Sleemans Cream Ale,Muffin - Carrot Individual Wrap
3,356,Butter - Unsalted,Cup - Translucent 7 Oz Clear,"Pepper - White, Ground",Sage - Ground,"Pork - Bacon, Double Smoked"
4,412,Table Cloth 81x81 White,"Pepper - Black, Whole",Beef - Top Sirloin,Butter - Unsalted,Wine - Blue Nun Qualitatswein


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [132]:
distances_df_cons = pd.DataFrame(1/(1 + squareform(pdist(data_pivot.T, 'kulsinski'))), 
                         index=data_pivot.columns, columns=data_pivot.columns)

customersID = list(data_pivot.columns)
def prod_recommend(customers):
    recommendations = dict()
    for cust in customers:
        similarities = distances_df_cons[cust].sort_values(ascending=False)[1:6]
        total_similar = pd.DataFrame(total_Q.loc[similarities.keys()])
        buy_products = total_similar.groupby('ProductName').sum().sort_values(by='Quantity',ascending=False)
        total = pd.concat([buy_products, total_Q.loc[cust]], axis=1).fillna(0)
        total.columns = ['Quantity Colegas', 'Quantity 33']
        products = list(total.sort_values(by= 'Quantity Colegas', ascending=False)[total['Quantity 33']==0].head().T.columns)
        recommendations[cust]=products
    return recommendations
step10 = prod_recommend(customersID)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  del sys.path[0]


In [134]:
step10_final = pd.DataFrame(step10).T.reset_index().head()
print('-----kulsinski-----')
display(step10_final)
print('-----euclidean-----')
display(step9)

-----kulsinski-----


,index,0,1,2,3,4
0,33,Beef - Ground Medium,Ketchup - Tomato,Cream Of Tartar,Spinach - Baby,Durian Fruit
1,200,Grouper - Fresh,Wine - White Cab Sauv.on,Pasta - Orecchiette,General Purpose Trigger,Langers - Ruby Red Grapfruit
2,264,Scallops - 10/20,Wine - Ej Gallo Sierra Valley,Pastry - Cheese Baked Scones,Pastry - Choclate Baked,Chicken - Wieners
3,356,"Cheese - Brie, Triple Creme",Chicken - Soup Base,Sauce - Hollandaise,"Pepper - White, Ground",Tea - Herbal Sweet Dreams
4,412,Scallops - 10/20,Bandage - Flexible Neon,Juice - Orange,"Turkey - Whole, Fresh",Cinnamon Buns Sticky


-----euclidean-----


,index,0,1,2,3,4
0,33,Chocolate - Dark,Wine - Ej Gallo Sierra Valley,Bay Leaf,Wine - Charddonnay Errazuriz,Pecan Raisin - Tarts
1,200,Scampi Tail,Muffin - Carrot Individual Wrap,Bay Leaf,Pork - Kidney,Soup - Campbells Bean Medley
2,264,Cheese - Parmesan Grated,Soup - Campbells Tomato Ravioli,Pork - Kidney,Beer - Sleemans Cream Ale,Muffin - Carrot Individual Wrap
3,356,Butter - Unsalted,Cup - Translucent 7 Oz Clear,"Pepper - White, Ground",Sage - Ground,"Pork - Bacon, Double Smoked"
4,412,Table Cloth 81x81 White,"Pepper - Black, Whole",Beef - Top Sirloin,Butter - Unsalted,Wine - Blue Nun Qualitatswein
